



# Determining Success and Failure Factors for Startups
***

## Introduction

The purpose of this project is to find out the success and failure factors behind startup companies. To that end, I am going to investigate a dataset provided by [Metric.am](https://metric.am/), containing inforation about 472 startups and their status: *'Success'* or *'Failed'*. 

## Methodology

For the sake of this project, we will consider two classes of startups: *failed* and *successful*. As already mentioned above, we are interested in the factors behind the success or failure of the startup. If we tried to map the problem to the data, we would be interested in the importance of each feature or attribute while predicting the status of the company. Thus, it would be most meaningful and comfortable to solve the problem using a model which would provide us with this information. 

A model that seems suitable under the mentioned constraints is Logstic Regression. It is a classification algorithm, so it can trained to predict the status of the company. It also associates weights with idividual features, which we can use to determine the importance of the feature, as well as point out if its effect is positive or negative.

So this point onwards, we are going to pursue the following steps:
- [Retrieve the data](#Retrieving-data)
- [Explore the data](#Exploring-data)
- [Prepare the data]()
- [Build the classifier]()
    - [Split the data into train-test sets]()
    - [Build the model]()
    - [Train the model]()
- [Test the accuracy]()
- [Analyze the calculated weights]()
- [Report the conclusions](#Report)

***
## Retrieving data

First of all, let's download the data into a *.csv* file. I will use [wget](https://www.gnu.org/software/wget/) to download raw data from the [GitHub repository](https://github.com/Metricam/Internship_tasks/tree/master/Startup_Success).

In [617]:
!wget -O data.csv https://raw.githubusercontent.com/Metricam/Internship_tasks/master/Startup_Success/data.csv

--2020-05-19 20:22:05--  https://raw.githubusercontent.com/Metricam/Internship_tasks/master/Startup_Success/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 315563 (308K) [text/plain]
Saving to: ‘data.csv’

100%[======================================>] 315,563     --.-K/s   in 0.02s   

2020-05-19 20:22:06 (14.2 MB/s) - ‘data.csv’ saved [315563/315563]



We are also provided a *dictionary* dataset, which explains some of the features provided in the main dataset. Let's get that too.

In [618]:
!wget -O dictionary.csv https://raw.githubusercontent.com/Metricam/Internship_tasks/master/Startup_Success/dictionary.csv

--2020-05-19 20:22:07--  https://raw.githubusercontent.com/Metricam/Internship_tasks/master/Startup_Success/dictionary.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5183 (5.1K) [text/plain]
Saving to: ‘dictionary.csv’

100%[======================================>] 5,183       --.-K/s   in 0s      

2020-05-19 20:22:07 (42.5 MB/s) - ‘dictionary.csv’ saved [5183/5183]



***
## Exploring data

Now let's understand the data. For that we will need to import some libraries. (Notice, you might need to install the libraries. If that's the case, un-comment the first four lines and then run the block.)

In [619]:
# import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import re

### About dataset

What are the attributes?

In [620]:
data_dict = pd.read_csv('dictionary.csv')
pd.options.display.max_rows = None
pd.options.display.max_columns = None
display(data_dict)

,Variable,Description
0,Company_Name,NaN
1,Dependent-Company Status,Dependent variable indicating if company succe...
2,year of founding,NaN
3,Age of company in years,NaN
4,Internet Activity Score,How much company is acgtive on social media
5,Short Description of company profile,NaN
6,Industry of company,NaN
7,Focus functions of company,NaN
8,Investors,List of investors
9,Employee Count,NaN


Let's take a look at our data. 

In [621]:
#df = pd.read_csv('data.csv')
#df.head()

The default encoder in read_csv() function is *UTF-8*, however, as we can see from the above error, the data in the *data.csv* file in not UTF-8 encoded. Let's try several other encodings to see if any of them succeeds to decode the data. Here are some: *latin1*, *iso-8859-1*, *cp1252*.

In [622]:
df = pd.read_csv('data.csv', encoding='latin1')
df.head()
#display(df)

,Company_Name,Dependent-Company Status,year of founding,Age of company in years,Internet Activity Score,Short Description of company profile,Industry of company,Focus functions of company,Investors,Employee Count,Employees count MoM change,Has the team size grown,Est. Founding Date,Last Funding Date,Last Funding Amount,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Presence of a top angel or venture fund in previous round of investment,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Specialization of highest education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Employee benefits and salary structures,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,Client Reputation,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Gartner hype cycle stage,Time to maturity of technology (in years),Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
0,Company1,Success,No Info,No Info,-1.0,Video di

Lucky for us, *latin1* was the real encoding!

***
## Preparing data

Now let's prepare our data for classification and separate the features from the labels. 

**Let's print the shape of our dataset before cleaning and hope that we'll not destroy it comletely after cleaning. :D**

In [623]:
print("Shape before cleaning: ", df.shape)

Shape before cleaning:  (472, 116)


### Choosing the feature set...

First of all, it seems like following columns don't give too much value for our purpose: *'Company_Name', 'Short Description of company profile', 'Last Funding Date', 'Est. Founding Date', 'Age of company in years', 'Gartner hype cycle stage', 'Time to maturity of technology (in years)'*, so let's drop them!

Also, the status column should not be in the feature set.

In [624]:
features = df.drop(['Dependent-Company Status', 'Company_Name', 'Short Description of company profile', 
                    'Last Funding Date', 'Est. Founding Date', 'Age of company in years', 
                    'Gartner hype cycle stage', 'Time to maturity of technology (in years)'], axis=1)
features.head()

,year of founding,Internet Activity Score,Industry of company,Focus functions of company,Investors,Employee Count,Employees count MoM change,Has the team size grown,Last Funding Amount,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Presence of a top angel or venture fund in previous round of investment,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Specialization of highest education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Employee benefits and salary structures,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,Client Reputation,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
0,No Info,-1.0,NaN,operation,KPCB Holdings|Draper Fisher Jurvetson (DFJ)|Kl...,3.0,0.0,No,450000.0,United States,North America,2,0,1,2,2,15,Yes,4,Nothing,No,Small,No,No,No,No,Service,No,Private,No,Both,Yes,Platform,Global,Linear,Yes,None,No,No,No,No,N

In [625]:
statuses = df['Dependent-Company Status']
statuses.head()

0    Success
1    Success
2    Success
3    Success
4    Success
Name: Dependent-Company Status, dtype: object

### Handling NaN values...

Before we rush to drop the rows that have at least one NaN value, let's see which features have the most NaN values. Maybe the presence of those features is less important compared to the amount of data we'll lose...

In [626]:
nan_df = pd.DataFrame(features.isna().sum().sort_values(ascending=False))
print(nan_df[nan_df[0] != 0].count())
nan_df[nan_df[0] != 0]

0    12
dtype: int64


,0
Employees count MoM change,205
Employee Count,166
Last Funding Amount,160
Investors,140
Industry of company,124
Specialization of highest education,97
Industry trend in investing,82
Continent of company,71
Country of company,71
Internet Activity Score,65


The overall picture is not that bad, from 116 features, only 12 have NaN values, the rest are either fully filled in or the NaN values are expressed verbally, like 'no info'. Let's now quickly go over the list and see if it makes sense to keep these features.

- **'Employees count MoM change'** makes nearly the half of the dataset drop, so let's just get rid of that feature, as I don't even know what MoM is (Ministry of Manpower?).
- **'Employee Count'** who doesn't have this record? It seems like such a  primitive question! Anyways, this is an important data, and, sadly, we have to drop 166 lines!
- **'Last Funding Amount'** let's just drop this column.
- **'Investors'** this one is different, my intuition is that the rows that don't have investors have NaN values, so instead of dropping them, I'm going to substitute those with empty strings. Later, when I will make this variable binary, those rows will just have 0s for all the Investors. 
- **'Industry of company'** is pretty important in my opinion, so I'm going to drop the rows that have NaN.
- **'Specialization of highest education'** is a crazy field: some have meaningful answers and some have things like _"PhD"_ or _"INDUSTRI"_ or _"computers"_. Plus, this field is not directly connected to the success, in my opinion, the connection between the field of the startup and the specialization is what matters. We could filter all the fields out from specialization in complex ways and then compare with the fields of the startup, but if we take a quick look at the dataset, we'll see that the answers vary a lot and maybe it's best in this case to just discard this column and not over-engineer.
- The rest are fairly important and don't have drastic results, so we'll just drom the NaN rows.

In [627]:
# Drop the discussed columns.
features = features.drop(['Employees count MoM change', 'Last Funding Amount', 
                          'Specialization of highest education'], axis=1)

# Sustitute NaNs with empty strings.
features['Investors'] = features['Investors'].apply(lambda x: x if isinstance(x, str) else '')

# Drop the rest of the rows.
features = features.dropna()

features.head()

,year of founding,Internet Activity Score,Industry of company,Focus functions of company,Investors,Employee Count,Has the team size grown,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Presence of a top angel or venture fund in previous round of investment,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Employee benefits and salary structures,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,Client Reputation,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
2,2011,455.0,Analytics|Cloud Computing|Software Development,operations,TechStars|Streamlined Ventures|Amplify Partner...,14.0,No,United States,North America,15,0,3,0,7,10,No,0,low,Yes,Medium,No,No,No,No,Both,Yes,Private,Yes,Both,Yes,cloud,Local,Non-Linear,No,Few,No,No,No,Yes,No,No,Yes,No,No,No,Online,B2B,Low,Medium,Yes,Low,Bachelors

In [628]:
features.shape

(239, 105)

In [629]:
display(features)

,year of founding,Internet Activity Score,Industry of company,Focus functions of company,Investors,Employee Count,Has the team size grown,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Presence of a top angel or venture fund in previous round of investment,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Employee benefits and salary structures,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,Client Reputation,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
2,2011,455.0,Analytics|Cloud Computing|Software Development,operations,TechStars|Streamlined Ventures|Amplify Partner...,14.0,No,United States,North America,15,0,3,0,7,10,No,0,low,Yes,Medium,No,No,No,No,Both,Yes,Private,Yes,Both,Yes,cloud,Local,Non-Linear,No,Few,No,No,No,Yes,No,No,Yes,No,No,No,Online,B2B,Low,Medium,Yes,Low,Bachelors

### Handling not NaN empty values...

Although we removed a bunch of rows with NaN values, there are still some records that in principle are not NaN but have the same effect. As those values can be anything, I am going to manually look into the dataset to detect them. To, nevertheless, ease the job, I'll look into the set of unique values of each column. 

In [630]:
col_sets = []
for c in features.columns:
    col_sets.append(c)
    col_sets.append(features[c].unique())
    
#col_sets
col_sets[20:26] # Just showing a sample batch.

['Number of Investors in Angel and or VC',
 array(['0', '2', '3', '6', '1', 'No Info', '4', '5', '7', '9', '8'],
       dtype=object),
 'Number of Co-founders',
 array([3, 2, 1, 4, 0, 5, 7, 6]),
 'Number of of advisors',
 array([ 0,  1,  9,  4,  8,  2,  6,  3,  7,  5, 11])]

 **Based on my obervations, here are the NaN value strings:**

- 'No Info' 
<!-- 'Number of Investors in Seed', 'Number of Investors in Angel and or VC', 'Presence of a top angel or venture fund in previous round of investment', 'year of founding', 'Team size all employees', 'Presence of a top angel or venture fund in previous round of investment', 'Subscription based business', 'Local or global player', 'Linear or Non-linear business model', 'Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive', 'Number of  of Partners of company', 'Online or offline venture - physical location based business or online venture?', 'Exposure across the globe', 'Renowned in professional circle', 'Breadth of experience across verticals', 'Highest education', 'Years of education', 'Relevance of education to venture', 'Relevance of experience to venture', 'Degree from a Tier 1 or Tier 2 university?', 'Employees per year of company existence', 'Time to 1st investment (in months)', 'Avg time to investment - average across all rounds, measured from previous investment', 'Team Composition score', 'Pricing Strategy', 'Time to market service or product', 'Employee benefits and salary structures', 'Long term relationship with other founders', 'Client Reputation', 'Solutions offered', 'Invested through global incubation competitions?', 'Survival through recession, based on existence of the company through recession times' -->
- '\\\\' 
- 'N'
- 'unknown amount'
 

Let's see what are the losses of cleaning in this case.

In [631]:
no_info_df = pd.DataFrame(features.apply(lambda x: (x == "No Info") | (x == "\\") 
                        | (x == "N") | (x == "unknown amount"))
                          .sum().sort_values(ascending=False))

print(no_info_df[no_info_df[0] != 0].count())
no_info_df[no_info_df[0] != 0]

0    36
dtype: int64


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,0
Employee benefits and salary structures,156
Client Reputation,103
Last round of funding received (in milionUSD),43
Presence of a top angel or venture fund in previous round of investment,36
google page rank of company website,26
Invested through global incubation competitions?,21
Years of education,13
Highest education,13
Employees per year of company existence,13
Number of of Partners of company,11


Let's drop the first 3 columns, as they cause major data loss, and clear the rest.

In [632]:
features = features.drop(['Employee benefits and salary structures', 'Client Reputation', 
                          'Presence of a top angel or venture fund in previous round of investment'], axis=1)

In [633]:
for col in features.columns:
    features = features[(features[col] != "No Info") & (features[col] != "\\") 
                        & (features[col] != "N") & (features[col] != "unknown amount")]
    
print(features.shape)
features = features.reset_index(drop=True)
features.head()

(118, 102)


,year of founding,Internet Activity Score,Industry of company,Focus functions of company,Investors,Employee Count,Has the team size grown,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
0,2011,455.0,Analytics|Cloud Computing|Software Development,operations,TechStars|Streamlined Ventures|Amplify Partner...,14.0,No,United States,North America,15,0,3,0,7,10,0,low,Yes,Medium,No,No,No,No,Both,Yes,Private,Yes,Both,Yes,cloud,Local,Non-Linear,No,Few,No,No,No,Yes,No,No,Yes,No,No,No,Online,B2B,Low,Medium,Yes,Low,Bachelors,18,Yes,Yes,Tier_2,500,High,0,0,1,Medium,18,None,36,High,Medium,Yes,No,Low,Yes,Yes,Yes,No,No,No,71391,Yes,Yes,Yes,3.0,Medium,0,3.3333

### Converting list values into binary variables...

Let's now convert the following values into lists, so that we can then make them binary variables.

 - *'Industry of company'*
 - *'Investors'*
 - *'Focus functions of company'*

In [634]:
features['Industry of company'] = features['Industry of company'].apply(lambda x: x.lower().strip().split('|'))
features['Investors'] = features['Investors'].apply(lambda x: x.lower().strip().split('|'))

def process_str(x) :
    str_list = re.split(',|&|\n|\|', x.lower())
    str_list = list(map(str.strip, str_list))
    return str_list

features['Focus functions of company'] = features['Focus functions of company'].apply(lambda x: process_str(x))

features.head()

,year of founding,Internet Activity Score,Industry of company,Focus functions of company,Investors,Employee Count,Has the team size grown,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
0,2011,455.0,"[analytics, cloud computing, software developm...",[operations],"[techstars, streamlined ventures, amplify part...",14.0,No,United States,North America,15,0,3,0,7,10,0,low,Yes,Medium,No,No,No,No,Both,Yes,Private,Yes,Both,Yes,cloud,Local,Non-Linear,No,Few,No,No,No,Yes,No,No,Yes,No,No,No,Online,B2B,Low,Medium,Yes,Low,Bachelors,18,Yes,Yes,Tier_2,500,High,0,0,1,Medium,18,None,36,High,Medium,Yes,No,Low,Yes,Yes,Yes,No,No,No,71391,Yes,Yes,Yes,3.0,Medium

In [635]:
# For every row in the dataframe, iterate through the list of industries 
# and place a 1 into the corresponding column.
for index, row in features.iterrows():
    for industry in row['Industry of company']:
        features.at[index, industry] = 1
        
# Filling in the NaN values with 0 to show that a company doesn't belong to that column's industry.
features = features.fillna(0)

# Same for investors.
for index, row in features.iterrows():
    for investor in row['Investors']:
        features.at[index, investor] = 1

features = features.fillna(0)

# And for focus functions of company.
for index, row in features.iterrows():
    for focus in row['Focus functions of company']:
        features.at[index, focus] = 1
        
features = features.fillna(0)
        
features.head()

,year of founding,Internet Activity Score,Industry of company,Focus functions of company,Investors,Employee Count,Has the team size grown,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score,analytics,cloud computing,software development,marketing,enterprise software,food & beverages,hospitality,network / hosting / infrastructure,mobile,healthcare,pharmaceuticals,media,finance,e-commerce,gaming,email,security,publishing,education,advertising,entertainment,transportation,retail,social networking,search,insurance,cleantech,energy,market research,deals,telecommunications,music,techstars,streamlined ventures,amplify partners,rincon venture partners,pe

And drop the original columns with lists.

In [636]:
features = features.drop('Industry of company', 1).drop('Investors', 1).drop('Focus functions of company', 1)
features.head()

,year of founding,Internet Activity Score,Employee Count,Has the team size grown,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score,analytics,cloud computing,software development,marketing,enterprise software,food & beverages,hospitality,network / hosting / infrastructure,mobile,healthcare,pharmaceuticals,media,finance,e-commerce,gaming,email,security,publishing,education,advertising,entertainment,transportation,retail,social networking,search,insurance,cleantech,energy,market research,deals,telecommunications,music,techstars,streamlined ventures,amplify partners,rincon venture partners,pelion venture partners,500 startups,loren siebert,jason se

### Encoding categorical variables as numericals...

First, let's encode the statuses.

In [637]:
statuses.unique()

array(['Success', 'Failed'], dtype=object)

In [638]:
statuses.replace(to_replace=['Success', 'Failed'], value=[1,0], inplace=True)
statuses.head()

0    1
1    1
2    1
3    1
4    1
Name: Dependent-Company Status, dtype: int64

Let's consider answers having less than 20 unique values as categorical in our feature list.

In [639]:
cat_columns = []
cat_vals = []
for col in features.columns:
    if len(features[col].unique()) < 21:
        if isinstance(features[col][0], str):
            col_set = sorted(features[col].apply(lambda x: x.lower()).unique())
        else:
            col_set = sorted(features[col].unique())
        if not((len(col_set) == 2) and (col_set[0] == 0) and (col_set[1] == 1)):
            cat_columns.append(col)
            cat_vals.append(col_set)
        
categoricals = pd.DataFrame()
categoricals['Feature'] = cat_columns
categoricals['Categories'] = cat_vals

categoricals

,Feature,Categories
0,year of founding,"[1999, 2000, 2002, 2004, 2005, 2006, 2007, 200..."
1,Has the team size grown,"[no, yes]"
2,Country of company,"[belgium, bulgaria, czech republic, denmark, f..."
3,Continent of company,"[asia, europe, north america]"
4,Number of Investors in Seed,"[0, 1, 10, 13, 15, 17, 2, 22, 24, 3, 4, 5, 6, ..."
5,Number of Investors in Angel and or VC,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
6,Number of Co-founders,"[0, 1, 2, 3, 4, 5, 7]"
7,Number of of advisors,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 11]"
8,Team size Senior leadership,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
9,Number of of repeat investors,"[0, 1, 10, 2, 3, 4]"


Let's take out all the features that have two options and replace them with 0/1s. This includes yes/no, linear/non-linear, local/global, etc..

In [640]:
for row, col in zip(range(len(categoricals)), categoricals.Feature):
    if isinstance(features[col][0], str):
        col_set = categoricals['Categories'][row]
        if (len(col_set) == 2):
            features[col] = features[col].apply(lambda x: x.lower())
            features[col].replace(to_replace=col_set, value=[0,1], inplace=True)
            categoricals = categoricals[categoricals['Feature'] != col]
features.head()

,year of founding,Internet Activity Score,Employee Count,Has the team size grown,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Controversial history of founder or co founder,Legal risk and intellectual property,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score,analytics,cloud computing,software development,marketing,enterprise software,food & beverages,hospitality,network / hosting / infrastructure,mobile,healthcare,pharmaceuticals,media,finance,e-commerce,gaming,email,security,publishing,education,advertising,entertainment,transportation,retail,social networking,search,insurance,cleantech,energy,market research,deals,telecommunications,music,techstars,streamlined ventures,amplify partners,rincon venture partners,pelion venture partners,500 startups,loren siebert,jason se

In [641]:
categoricals = categoricals.reset_index(drop=True)
categoricals

,Feature,Categories
0,year of founding,"[1999, 2000, 2002, 2004, 2005, 2006, 2007, 200..."
1,Country of company,"[belgium, bulgaria, czech republic, denmark, f..."
2,Continent of company,"[asia, europe, north america]"
3,Number of Investors in Seed,"[0, 1, 10, 13, 15, 17, 2, 22, 24, 3, 4, 5, 6, ..."
4,Number of Investors in Angel and or VC,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
5,Number of Co-founders,"[0, 1, 2, 3, 4, 5, 7]"
6,Number of of advisors,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 11]"
7,Team size Senior leadership,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
8,Number of of repeat investors,"[0, 1, 10, 2, 3, 4]"
9,Number of Sales Support material,"[high, low, medium, nothing]"


As we can see, the *'Controversial history of founder or co founder'* feature has a unique answer throughout the database, so let's just drop it, as it does not give us any information.

In [642]:
features = features.drop('Controversial history of founder or co founder', 1)
categoricals = categoricals[categoricals['Feature'] != 'Controversial history of founder or co founder']

categoricals

,Feature,Categories
0,year of founding,"[1999, 2000, 2002, 2004, 2005, 2006, 2007, 200..."
1,Country of company,"[belgium, bulgaria, czech republic, denmark, f..."
2,Continent of company,"[asia, europe, north america]"
3,Number of Investors in Seed,"[0, 1, 10, 13, 15, 17, 2, 22, 24, 3, 4, 5, 6, ..."
4,Number of Investors in Angel and or VC,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
5,Number of Co-founders,"[0, 1, 2, 3, 4, 5, 7]"
6,Number of of advisors,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 11]"
7,Team size Senior leadership,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
8,Number of of repeat investors,"[0, 1, 10, 2, 3, 4]"
9,Number of Sales Support material,"[high, low, medium, nothing]"


Let's now rule out the Features that don't seem feasible to be viewed as categoricals. These features are:

- 'Percent_skill_Consulting'
- 'Percent_skill_Finance'
- 'Renowned in professional circle'

The rest are better off as categoricals, as the dependency should not neccessarily be linear, and we can better analyze having those features as categoricals.

In [643]:
categoricals = categoricals[categoricals['Feature'] != 'Percent_skill_Consulting']
categoricals = categoricals[categoricals['Feature'] != 'Percent_skill_Finance']
categoricals = categoricals[categoricals['Feature'] != 'Renowned in professional circle']
categoricals = categoricals.reset_index(drop=True)

categoricals

,Feature,Categories
0,year of founding,"[1999, 2000, 2002, 2004, 2005, 2006, 2007, 200..."
1,Country of company,"[belgium, bulgaria, czech republic, denmark, f..."
2,Continent of company,"[asia, europe, north america]"
3,Number of Investors in Seed,"[0, 1, 10, 13, 15, 17, 2, 22, 24, 3, 4, 5, 6, ..."
4,Number of Investors in Angel and or VC,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
5,Number of Co-founders,"[0, 1, 2, 3, 4, 5, 7]"
6,Number of of advisors,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 11]"
7,Team size Senior leadership,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
8,Number of of repeat investors,"[0, 1, 10, 2, 3, 4]"
9,Number of Sales Support material,"[high, low, medium, nothing]"


Great! Now we are left only with the features that should be encoded as binary variables, like we did in case of lists. We just ahve to make sure that every column has the original feature name attached to the answer so that noting gets overwrote (e.g. *'both'*, *'none'*). 

In [644]:
for index, ignored in features.iterrows():
    for row, feature in zip(range(len(categoricals)), categoricals.Feature):
        for option in categoricals['Categories'][row]:
            col_name = feature + " " + str(option)
            features.at[index, col_name] = 1
        
features = features.fillna(0)

features.head()

,year of founding,Internet Activity Score,Employee Count,Has the team size grown,Country of company,Continent of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of Co-founders,Number of of advisors,Team size Senior leadership,Team size all employees,Number of of repeat investors,Number of Sales Support material,Worked in top companies,Average size of companies worked for in the past,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Product or service company?,Catering to product/service across verticals,Focus on private or public data?,Focus on consumer data?,Focus on structured or unstructured data,Subscription based business,Cloud or platform based serive/product?,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Number of of Partners of company,Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,Online or offline venture - physical location based business or online venture?,B2C or B2B venture?,Top forums like 'Tech crunch' or 'Venture beat' talking about the company/model - How much is it being talked about?,Average Years of experience for founder and co founder,Exposure across the globe,Breadth of experience across verticals,Highest education,Years of education,Relevance of education to venture,Relevance of experience to venture,Degree from a Tier 1 or Tier 2 university?,Renowned in professional circle,Experience in selling and building products,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Top management similarity,Number of Recognitions for Founders and Co-founders,Number of of Research publications,Skills score,Team Composition score,Dificulty of Obtaining Work force,Pricing Strategy,Hyper localisation,Time to market service or product,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Legal risk and intellectual property,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Industry trend in investing,Disruptiveness of technology,Number of Direct competitors,Employees per year of company existence,Last round of funding received (in milionUSD),"Survival through recession, based on existence of the company through recession times",Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score,analytics,cloud computing,software development,marketing,enterprise software,food & beverages,hospitality,network / hosting / infrastructure,mobile,healthcare,pharmaceuticals,media,finance,e-commerce,gaming,email,security,publishing,education,advertising,entertainment,transportation,retail,social networking,search,insurance,cleantech,energy,market research,deals,telecommunications,music,techstars,streamlined ventures,amplify partners,rincon venture partners,pelion venture partners,500 startups,loren siebert,jason seats,xg ventures,george karidis,sam choi,morris 

Let's drop the original columns.

In [645]:
features = features.reset_index(drop=True)

for feature in categoricals['Feature']:
    features = features.drop(feature, 1)
    
features.head()

,Internet Activity Score,Employee Count,Has the team size grown,Team size all employees,Worked in top companies,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Catering to product/service across verticals,Focus on consumer data?,Subscription based business,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,B2C or B2B venture?,Exposure across the globe,Relevance of education to venture,Relevance of experience to venture,Renowned in professional circle,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Number of Recognitions for Founders and Co-founders,Skills score,Pricing Strategy,Hyper localisation,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Legal risk and intellectual property,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Employees per year of company existence,Last round of funding received (in milionUSD),Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,analytics,cloud computing,software development,marketing,enterprise software,food & beverages,hospitality,network / hosting / infrastructure,mobile,healthcare,pharmaceuticals,media,finance,e-commerce,gaming,email,security,publishing,education,advertising,entertainment,transportation,retail,social networking,search,insurance,cleantech,energy,market research,deals,telecommunications,music,techstars,streamlined ventures,amplify partners,rincon venture partners,pelion venture partners,500 startups,loren siebert,jason seats,xg ventures,george karidis,sam choi,morris wheeler,data collective,pejman nozad,ullas naik,dirk elmendorf,galvanize,pat matthews,paul kedrosky,matt ocko,cloud power capital,jared kopf,anne johnson,issac roth,george karutz,jim deters,zachary aarons,zack bogue,dfj frontier,draper nexus ventures,gil elbaz,auren hoffman,walter kortschak,mi ventures,brand ventures,daher capital,double m partners,gold hill capital,clark landry,draper associates,mi ventures llc,signia venture partners,pritzker group venture capital,excelerate labs,hyde park venture partners,chicago ventures,amicus capital,ideo,olive ventures,kd capital l.l.c,norwest venture partners,bessemer venture partners,atlas venture,promus ventures,softtech vc,costanoa venture capital,lee linden,chamath palihapitiya,raj de datta,tim kendall,omar siddiqui,david vivero,sequoia capital,khosla ventures,redpoint ventures,first round capital,pivotnorth capital,battery ventures,in-q-tel,jump capital,sap ventures,northwestern university,harvard business school angels,tech coast angels,hummer winblad venture partners,us venture partners,presidio ventures,mohr davidow ventures,grotech ventures,access venture partners,ffp holdings,matrix partners,allan zeise,thomas shannon,allen zeise,wisconsin investment partners,silicon

Now let's make sure everything is a float type in our dataset.

In [646]:
for col in features.columns:
    try:
        features[col] = features[col].apply(lambda x: float(x))
    except:
        pass

features.head()

,Internet Activity Score,Employee Count,Has the team size grown,Team size all employees,Worked in top companies,Have been part of startups in the past?,Have been part of successful startups in the past?,Was he or she partner in Big 5 consulting?,Consulting experience?,Catering to product/service across verticals,Focus on consumer data?,Subscription based business,Local or global player,Linear or Non-linear business model,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Crowdsourcing based business,Crowdfunding based business,Machine Learning based business,Predictive Analytics business,Speech analytics business,Prescriptive analytics business,Big Data Business,Cross-Channel Analytics/ marketing channels,Owns data or not? (monetization of data) e.g. Factual,Is the company an aggregator/market place? e.g. Bluekai,B2C or B2B venture?,Exposure across the globe,Relevance of education to venture,Relevance of experience to venture,Renowned in professional circle,Experience in Fortune 100 organizations,Experience in Fortune 500 organizations,Experience in Fortune 1000 organizations,Number of Recognitions for Founders and Co-founders,Skills score,Pricing Strategy,Hyper localisation,Long term relationship with other founders,Proprietary or patent position (competitive position),Barriers of entry for the competitors,Company awards,Legal risk and intellectual property,google page rank of company website,Technical proficiencies to analyse and interpret unstructured data,Solutions offered,Invested through global incubation competitions?,Employees per year of company existence,Last round of funding received (in milionUSD),Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment",Percent_skill_Entrepreneurship,Percent_skill_Operations,Percent_skill_Engineering,Percent_skill_Marketing,Percent_skill_Leadership,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,analytics,cloud computing,software development,marketing,enterprise software,food & beverages,hospitality,network / hosting / infrastructure,mobile,healthcare,pharmaceuticals,media,finance,e-commerce,gaming,email,security,publishing,education,advertising,entertainment,transportation,retail,social networking,search,insurance,cleantech,energy,market research,deals,telecommunications,music,techstars,streamlined ventures,amplify partners,rincon venture partners,pelion venture partners,500 startups,loren siebert,jason seats,xg ventures,george karidis,sam choi,morris wheeler,data collective,pejman nozad,ullas naik,dirk elmendorf,galvanize,pat matthews,paul kedrosky,matt ocko,cloud power capital,jared kopf,anne johnson,issac roth,george karutz,jim deters,zachary aarons,zack bogue,dfj frontier,draper nexus ventures,gil elbaz,auren hoffman,walter kortschak,mi ventures,brand ventures,daher capital,double m partners,gold hill capital,clark landry,draper associates,mi ventures llc,signia venture partners,pritzker group venture capital,excelerate labs,hyde park venture partners,chicago ventures,amicus capital,ideo,olive ventures,kd capital l.l.c,norwest venture partners,bessemer venture partners,atlas venture,promus ventures,softtech vc,costanoa venture capital,lee linden,chamath palihapitiya,raj de datta,tim kendall,omar siddiqui,david vivero,sequoia capital,khosla ventures,redpoint ventures,first round capital,pivotnorth capital,battery ventures,in-q-tel,jump capital,sap ventures,northwestern university,harvard business school angels,tech coast angels,hummer winblad venture partners,us venture partners,presidio ventures,mohr davidow ventures,grotech ventures,access venture partners,ffp holdings,matrix partners,allan zeise,thomas shannon,allen zeise,wisconsin investment partners,silicon

To be even surer, lets print the data types:

In [647]:
features.dtypes


Internet Activity Score                                                                                                              float64
Employee Count                                                                                                                       float64
Has the team size grown                                                                                                              float64
Team size all employees                                                                                                              float64
Worked in top companies                                                                                                              float64
Have been part of startups in the past?                                                                                              float64
Have been part of successful startups in the past?                                                                                   float64
Was he or she

#### FABULOUS! 

### Normalizing the data...

Let's now normalize the data, so that our analysis are accurate.

In [648]:
X = preprocessing.StandardScaler().fit(features).transform(features)
X[0:7]

array([[ 1.29210852, -0.34979314, -0.9029865 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.47613575,  0.11649234, -0.9029865 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.27436718, -0.34979314, -0.9029865 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.62332663, -0.3124903 , -0.9029865 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.51225604, -0.55495875, -0.9029865 , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.67006852,  0.02323524, -0.9029865 , ...,  0.        ,
         0.        ,  0.        ]])

In [649]:
y = statuses.values
y[0:7]

array([1, 1, 1, 1, 1, 1, 1])

***
## Classification

### Train-test split

### Building the model

### Training

***
## Testing accuracy

***
## Analyzing results

***
## Report

***
## Thank you!

**Author:** [Aneta Baloyan](https://www.linkedin.com/in/aneta-baloyan/)

Email: *aneta.baloyan@gmail.com*

***

May 2020

